# Gemini 3.0 Pro 图片生成 API 使用指南

本 Notebook 演示如何通过 **七牛 AIToken** 平台，使用 **OpenAI SDK** 调用 **Gemini 3.0 Pro Image Preview** 模型生成图片。

## 模型简介

Gemini 3.0 Pro Image Preview 是 Google 推出的多模态模型，支持根据文本提示词生成高质量图片。通过七牛 AIToken 平台，可以使用 OpenAI 兼容接口轻松调用。

- **文本生成图片**：根据自然语言描述生成图片
- **图片编辑（图生图）**：基于输入图片进行编辑、风格转换等操作，支持单图和多图输入
- **自定义宽高比**：支持 `1:1`、`2:3`、`3:2`、`3:4`、`4:3`、`4:5`、`5:4`、`9:16`、`16:9`、`21:9` 等多种比例
- **多种分辨率**：支持 `1K`、`2K`、`4K` 三种分辨率

**API 端点**：`https://api.qnaigc.com/v1/chat/completions`  
**模型名称**：`gemini-3.0-pro-image-preview`

## 1. 环境准备

首先安装 `openai` 依赖包，并设置 API Key 和七牛 AIToken 服务地址。

In [ ]:
# 安装 openai SDK
%pip install openai -q

In [ ]:
import os
from openai import OpenAI

# 初始化 OpenAI 客户端，指向七牛 AIToken 平台
client = OpenAI(
    api_key=os.environ.get("QINIU_API_KEY", "<your-api-key>"),
    base_url="https://api.qnaigc.com/v1",
)

## 2. 基础用法 — 文本生成图片

使用 Chat Completions 接口，通过文本提示词生成图片。返回结果中 `message.images` 数组包含生成的图片（base64 data URL 格式）。

响应结构示例：
```json
{
  "choices": [{
    "message": {
      "content": "",
      "images": [{
        "type": "image_url",
        "image_url": { "url": "data:image/png;base64,..." },
        "index": 0
      }]
    }
  }]
}
```

In [ ]:
import base64
from IPython.display import Image, display


def extract_and_display_image(response):
    """从响应中提取图片并展示"""
    try:
        message = response.choices[0].message

        # 展示推理过程（如果有）
        if hasattr(message, "reasoning_content") and message.reasoning_content:
            print(f"reasoning_content: {message.reasoning_content}\n")

        # images 是非标准 OpenAI 字段，Pydantic v2 extra="allow" 允许直接属性访问
        if hasattr(message, "images") and message.images:
            all_image_data = []
            for img in message.images:
                data_url = img["image_url"]["url"]
                # 从 data URL 中分离出 base64 部分
                if "," in data_url:
                    image_data = base64.b64decode(data_url.split(",", 1)[1])
                    display(Image(data=image_data))
                    all_image_data.append(image_data)
            # 打印文本内容（如果有）
            if message.content:
                print(f"模型回复: {message.content}")
            return all_image_data[0] if len(all_image_data) == 1 else all_image_data

        # 没有图片，打印文本内容
        if message.content:
            print(message.content)
        return None
    except (IndexError, KeyError, AttributeError) as e:
        raise ValueError(f"响应解析失败：{e}")

In [ ]:
# 基础调用示例：生成一只可爱的橘猫
response = client.chat.completions.create(
    model="gemini-3.0-pro-image-preview",
    stream=False,
    messages=[
        {
            "role": "user",
            "content": "画一只可爱的橘猫",
        }
    ],
    extra_body={
        "image_config": {
            "aspect_ratio": "16:9",
            "image_size": "4K",
        }
    },
)

# 展示生成的图片
image_data = extract_and_display_image(response)

## 3. 参数说明

### 请求参数

| 参数 | 类型 | 说明 |
|------|------|------|
| `model` | string | 模型名称，使用 `gemini-3.0-pro-image-preview` |
| `messages` | array | 消息列表，包含 `role` 和 `content` |
| `stream` | boolean | 是否流式返回，图片生成建议设为 `false` |
| `image_config` | object | 图像配置对象，用于控制图像比例和分辨率（使用 OpenAI SDK 时需要通过 `extra_body` 传递） |

### image_config 参数

| 参数 | 类型 | 说明 |
|------|------|------|
| `aspect_ratio` | string | 图像宽高比，可选值：`1:1`、`2:3`、`3:2`、`3:4`、`4:3`、`4:5`、`5:4`、`9:16`、`16:9`、`21:9` |
| `image_size` | string | 图像分辨率，可选值：`1K`、`2K`、`4K` |

## 4. 更多示例

In [ ]:
# 示例：生成风景图片（1:1 比例）
response = client.chat.completions.create(
    model="gemini-3.0-pro-image-preview",
    stream=False,
    messages=[
        {
            "role": "user",
            "content": "画一幅宁静的日式庭院，樱花树下有一条石板小路，远处是朦胧的山峦，水彩画风格",
        }
    ],
    extra_body={
        "image_config": {
            "aspect_ratio": "1:1",
            "image_size": "2K",
        }
    },
)

image_data = extract_and_display_image(response)

In [ ]:
# 示例：生成竖版海报（9:16 比例）
response = client.chat.completions.create(
    model="gemini-3.0-pro-image-preview",
    stream=False,
    messages=[
        {
            "role": "user",
            "content": "设计一张赛博朋克风格的城市夜景海报，霓虹灯闪烁，雨中的街道倒映着五彩灯光",
        }
    ],
    extra_body={
        "image_config": {
            "aspect_ratio": "9:16",
            "image_size": "4K",
        }
    },
)

image_data = extract_and_display_image(response)

## 5. 图生图 — 单图编辑

通过在 `content` 中同时传入文本指令和图片 URL，可以基于已有图片进行编辑。图片通过 `image_url` 类型的内容块传入。

请求结构示例：
```json
{
  "messages": [{
    "role": "user",
    "content": [
      {"type": "text", "text": "把这张图片改成红色"},
      {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
    ]
  }]
}
```

In [ ]:
# 示例：单图编辑 — 将图片颜色改为红色
response = client.chat.completions.create(
    model="gemini-3.0-pro-image-preview",
    stream=False,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Change this image to red.",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://aitoken-public.qnaigc.com/example/generate-image/image-to-image-1.jpg"
                    },
                },
            ],
        }
    ],
)

image_data = extract_and_display_image(response)

## 6. 图生图 — 多图编辑

支持在同一请求中传入多张图片，模型会综合理解所有图片内容并根据文本指令进行处理。常见场景包括：图片融合、风格迁移、多图对比编辑等。

只需在 `content` 数组中添加多个 `image_url` 类型的内容块即可。

In [ ]:
# 示例：多图编辑 — 将两张图片融合为一张
response = client.chat.completions.create(
    model="gemini-3.0-pro-image-preview",
    stream=False,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Merge these two images into one, combining the elements of both images harmoniously.",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://aitoken-public.qnaigc.com/example/generate-image/image-to-image-1.jpg"
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://aitoken-public.qnaigc.com/example/generate-video/running-man.jpg"
                    },
                },
            ],
        }
    ],
)

image_data = extract_and_display_image(response)

## 7. 保存图片到本地

In [ ]:
# 将生成的图片保存到本地
if image_data:
    output_path = "output.png"
    with open(output_path, "wb") as f:
        f.write(image_data)
    print(f"图片已保存到: {output_path}")

## 更多资源

- [七牛 AIToken 文档](https://developer.qiniu.com/aitokenapi)
- [OpenAI Python SDK 文档](https://github.com/openai/openai-python)
- [七牛 API 调用示例文档](https://apidocs.qnaigc.com/417911128e0)